In [26]:
import utils
import datetime


In [27]:
# START_DATE = datetime.datetime(2024, 10, 1)
START_DATE = datetime.datetime(2025, 1, 1)
# END_DATE = datetime.datetime(2024, 12, 31)
END_DATE = datetime.datetime(2025, 7, 31)
BENEFICIARY = "UMONS - NUMÉDIART"

CONVENTION_NUMBER = "SleepSense -  DIFST 2380067"
RESEARCHER_FIRST_NAME = "Vincent"
RESEARCHER_LAST_NAME = "Stragier"
RESEARCHER_JOB_TITLE = "assistant de recherche"
CONVENTION_OCCUPATION = 100  # %
HOURS_PER_DAY_MIN = 7.6  # h
HOURS_PER_DAY_MAX = 9  # h
TIME_INCREMENT = 0.5  # h
UNDER_MIN_PROBABILITY = 0.05
FILLING_DATE = datetime.datetime.now(tz=datetime.timezone.utc)

researcher_holidays = [
    datetime.datetime(2024, 10, 14),
    datetime.datetime(2025, 1, 6),
]


In [28]:
years = list(range(START_DATE.year, END_DATE.year + 1))

years_and_files_to_generate = [
    (year, f"{year}___{BENEFICIARY}___{CONVENTION_NUMBER}___{RESEARCHER_LAST_NAME}_{RESEARCHER_FIRST_NAME}.xlsx".replace(" ", "_")) for year in years
]

years_and_files_to_generate


[(2025,
  '2025___UMONS_-_NUMÉDIART___SleepSense_-__DIFST_2380067___Stragier_Vincent.xlsx')]

In [29]:
import random


def get_description(
    date: datetime.datetime,
    first_day: int,
    last_day: int,
    year: int,
    researcher_holidays: list,
):
    """Generate the description for a given date.

    Args:
        date (datetime.datetime): The date to generate the description for.
        first_day (int): The first day of the month.
        last_day (int): The last day of the month.
        year (int): The year of the date.
        researcher_holidays (list): The list of researcher holidays.

    Returns:
        str: The description for the given date.
    """
    weekends, work_holidays, umons_holidays = utils.get_weekends_holidays_and_umons_holidays_for_year(
        year)

    name = utils.get_holiday_name(date)

    if last_day < date.day or first_day > date.day:
        return "Hors convention", day in weekends

    if date in weekends:
        return "Weekend", True

    if date in work_holidays:
        return f"JF - {name}", day in weekends

    if date in umons_holidays:
        if name.startswith("Récupération") or name.startswith("Fête de la Communauté française"):
            return f"{name}", day in weekends
        return f"Congé UMONS - {name}", day in weekends

    if date in researcher_holidays:
        return "Congé chercheur", day in weekends

    return "", day in weekends


# def get_random_hours_per_day(min_hours: float, max_hours: float, under_min_probability: float):
#     """Get a random number of hours per day.

#     Args:
#         min_hours (float): The minimum number of hours per day.
#         max_hours (float): The maximum number of hours per day.
#         under_min_probability (float): The probability of getting a number of hours per day under the minimum.

#     Returns:
#         float: The number of hours per day.
#     """
#     duration = random.uniform(min_hours, max_hours)
#     half_probability = under_min_probability / 2

#     if not (half_probability < random.random() < 1 - half_probability) and under_min_probability > 0:
#         return (min_hours / max_hours) * 0.99 * duration

#     return duration


# def get_work_days_duration(
#     number_of_days: int,
#     min_hours: float,
#     max_hours: float,
#     quota: float = 1,
#     under_min_probability: float = 0.05,
# ) -> list:
#     """Get the duration of work days.

#     Args:
#         number_of_days (int): The number of days to get the duration for.
#         min_hours (float): The minimum number of hours per day.
#         max_hours (float): The maximum number of hours per day.
#         under_min_probability (float): The probability of getting a number of hours per day under the minimum.

#     Returns:
#         list: The list of durations for the work days.
#     """

#     mean_duration = 0
#     while mean_duration < min_hours + 0.1 * (max_hours - min_hours):
#         durations = [get_random_hours_per_day(
#             min_hours, max_hours, under_min_probability) for _ in range(number_of_days)]
#         mean_duration = sum(durations) / number_of_days

#     return durations

def get_random_hours_per_day(min_hours: float, max_hours: float, under_min_probability: float):
    """Get a random number of hours per day.

    Args:
        min_hours (float): The minimum number of hours per day.
        max_hours (float): The maximum number of hours per day.
        under_min_probability (float): The probability of getting a number of hours per day under the minimum.

    Returns:
        float: The number of hours per day.
    """
    duration = random.uniform(min_hours, max_hours)
    half_probability = under_min_probability / 2

    if not (half_probability < random.random() < 1 - half_probability) and under_min_probability > 0:
        return (min_hours / max_hours) * 0.99 * duration

    return duration


def get_work_days_duration(
    number_of_days: int,
    min_hours: float,
    max_hours: float,
    quota: float = 1,
    under_min_probability: float = 0.05,
    time_increment: float = 0.5,
) -> list:
    """Get the duration of work days.

    Args:
        number_of_days (int): The number of days to get the duration for.
        min_hours (float): The minimum number of hours per day.
        max_hours (float): The maximum number of hours per day.
        quota (float): The quota of the researcher.
        under_min_probability (float): The probability of getting a number of hours per day under the minimum.
        time_increment (float): The time increment.

    Returns:
        list: The list of durations for the work days.
    """

    def _hours_per_day():
        hours_per_day = [
            get_random_hours_per_day(
                min_hours,
                max_hours,
                under_min_probability,
            ) for _ in range(number_of_days)
        ]

        hours_per_day = [round((hour * quota) / time_increment) *
                         time_increment for hour in hours_per_day]

        return hours_per_day

    mean_duration = 0
    while mean_duration < (min_hours + 0.1 * (max_hours - min_hours)) * quota:
        durations = _hours_per_day()
        mean_duration = sum(durations) / number_of_days

    return durations


def get_number_of_working_day(year: int, month: int, first_day: int, last_day: int, researcher_holidays: list):
    """Get the number of working days in a month.

    Args:
        year (int): The year of the month.
        month (int): The month to get the number of working days for.
        first_day (int): The first day of the month.
        last_day (int): The last day of the month.
        weekends (list): The list of weekends.
        holidays (list): The list of holidays.

    Returns:
        int: The number of working days in the month.
    """
    number_of_days = 0

    weekends, work_holidays, umons_holidays = utils.get_weekends_holidays_and_umons_holidays_for_year(
        year)

    for day in range(first_day, last_day + 1):
        current_date = datetime.datetime(year, month, day)
        if current_date in weekends:
            continue

        if current_date in work_holidays:
            continue

        if current_date in umons_holidays:
            continue

        if current_date in researcher_holidays:
            continue

        number_of_days += 1

    return number_of_days


content_per_file = {}

for year, file_template in years_and_files_to_generate:
    start_date = datetime.datetime(year, 1, 1)
    if year == START_DATE.year:
        start_date = START_DATE

    end_date = datetime.datetime(year, 12, 31)
    if year == END_DATE.year:
        end_date = END_DATE

    lines_in_month = {}
    for month in range(start_date.month, end_date.month + 1):
        month_str = f"{month:02d}"
        number_of_days = utils.number_of_days_in_month(year, month)

        first_day = 1
        if year == start_date.year and month == start_date.month:
            first_day = start_date.day

        last_day = number_of_days
        if year == end_date.year and month == end_date.month:
            last_day = end_date.day

        number_of_working_days = get_number_of_working_day(
            year,
            month,
            first_day,
            last_day,
            researcher_holidays,
        )

        work_hours_per_day = get_work_days_duration(
            number_of_working_days,
            HOURS_PER_DAY_MIN,
            HOURS_PER_DAY_MAX,
            CONVENTION_OCCUPATION / 100,
            UNDER_MIN_PROBABILITY,
        )

        lines_in_month[month_str] = []
        for day in range(1, number_of_days + 1):
            current_date = datetime.datetime(year, month, day)
            description, day_in_weekend = get_description(
                current_date,
                first_day,
                last_day,
                year,
                researcher_holidays
            )

            work_hours = ""
            if description == "":
                work_hours = work_hours_per_day.pop(0)

            line = {
                "day": day,
                "date": current_date.strftime("%d/%m/%Y"),
                "research_time": work_hours,
                "non_research_time": "",
                "description": description,
                "day_in_weekend": day_in_weekend,
            }
            lines_in_month[month_str].append(line)

    content_per_file[file_template] = {
        "year": year,
        "sheets_content": lines_in_month,
        "convention_beneficiary": BENEFICIARY,
        "convention_number": CONVENTION_NUMBER,
        "researcher_first_name": RESEARCHER_FIRST_NAME,
        "researcher_last_name": RESEARCHER_LAST_NAME,
        "researcher_job_title": RESEARCHER_JOB_TITLE,
        "convention_occupation": CONVENTION_OCCUPATION,
    }
    # print(lines_in_month)


In [30]:
# content_per_file
#

# Open the template file
import openpyxl
import pathlib
import shutil
template_file = pathlib.Path("templates/template_spw.xlsx")


# template_content = openpyxl.load_workbook(
#     template_file, read_only=False, rich_text=True)

for file_template, content in content_per_file.items():
    # Copy the template file (as a file).
    destination_file = pathlib.Path("generated_timesheets") / file_template
    destination_file.parent.mkdir(parents=True, exist_ok=True)

    if not destination_file.exists():
        shutil.copy(template_file, destination_file)

    destination_workbook = openpyxl.load_workbook(
        destination_file, read_only=False, rich_text=True)

    sheets_to_keep = list(content["sheets_content"].keys())
    for sheet in destination_workbook.sheetnames:
        if sheet not in sheets_to_keep:
            destination_workbook.remove(destination_workbook[sheet])
            continue

        sheet_content = content["sheets_content"][sheet]
        sheet = destination_workbook[sheet]

        sheet["C3"].value = content["convention_beneficiary"]
        sheet["C4"].value = content["convention_number"]
        sheet["C5"].value = f"{content['researcher_first_name']} {content['researcher_last_name']}"
        sheet["D6"].value = f"{content['convention_occupation']} %"

        sheet["H3"].value = f"{content['year']}"

        for index, row in enumerate(sheet.iter_rows(min_row=39, max_row=50, min_col=1, max_col=6), start=9):
            for cell in row:
                if cell.value == "Date:":
                    sheet[f"D{cell.row}"].value = FILLING_DATE.strftime(
                        "%d/%m/%Y")

                if str(cell.value).startswith('=C5&",'):
                    cell.value = f'=C5&", {content["researcher_job_title"]}"'

        # raise Exception("STOP")

        for index, line in enumerate(sheet_content, start=9):
            initial_font = sheet[f"F{index}"].font
            initial_content = sheet[f"F{index}"].value

            research_time = line["research_time"]
            if isinstance(research_time, (float, int)):
                research_time = round(research_time, 2)

            non_research_time = line["non_research_time"]
            if isinstance(non_research_time, (float, int)):
                non_research_time = round(non_research_time, 2)

            sheet[f"A{index}"].value = line["day"]
            sheet[f"B{index}"].value = line["date"]
            sheet[f"C{index}"].value = research_time
            sheet[f"D{index}"].value = non_research_time
            sheet[f"F{index}"].value = line["description"]

            if line["day_in_weekend"]:
                sheet[f"F{index}"].font = openpyxl.styles.Font(name=initial_font.name, size=initial_font.size,
                                                               color="FF0000", bold=True)
            elif line["description"] != "":
                sheet[f"F{index}"].font = openpyxl.styles.Font(name=initial_font.name, size=initial_font.size,
                                                               color="FF0000", bold=False, italic=True)
            else:
                if initial_content is not None and initial_content != "":
                    sheet[f"F{index}"].value = initial_content
                sheet[f"F{index}"].font = openpyxl.styles.Font(name=initial_font.name, size=initial_font.size,
                                                               color="000000", bold=False)

    destination_workbook.save(destination_file)
    destination_workbook.close()


In [ ]:
get_work_days_duration(20, 7.6, 9, 0.5, 0.05)


[4.5,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.5,
 4.5,
 4.5,
 4.0,
 4.5,
 4.5,
 3.5,
 4.5,
 4.0,
 4.5,
 4.5,
 4.0]